In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
import os
import pandas as pd

path="TrainingRatings.txt"

TrainingRatings = pd.read_csv(path, header=None, names=['movieId', 'customerId', 'rating'])


In [3]:
df_movie_features = TrainingRatings.pivot(
    index='customerId',
    columns='movieId',
    values='rating'
)
df_movie_features_new = df_movie_features.fillna(0)


In [4]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
nbrs = NearestNeighbors(n_neighbors=15, algorithm='ball_tree', metric='euclidean').fit(df_movie_features_new)
# distances, indices = nbrs.kneighbors([df_movie_features_new.iloc[0]])
# print(indices)

[[    0 26442 15836  9801 14582 10212 11948 17213 27282 24451 21741  3715
  10683  6161 10777]]


In [5]:
def func(ele):
    return ele[1]

def nn(userId):

  #Calculating 15 nearest neighbours for each user
  distances, indices = nbrs.kneighbors([df_movie_features_new.loc[userId]])

  A=df_movie_features.iloc[indices[0]]
  
  #calculating mean of a single user row for all movies
  m_sum = np .sum(np.matrix(df_movie_features_new.iloc[indices[0]]), axis=1, dtype='float')
  m_count = np.count_nonzero(np.array(df_movie_features_new.iloc[indices[0]]), axis=1, keepdims=True)

  # User row of all movies - User mean
  A_mA = A - (m_sum/m_count)
  
  #filling each NAN value by zero
  A_mA = A_mA.fillna(0)

  # Sum of squares across rows
  ssA = (A_mA**2).sum(1)

  #Numerator calculation of perason correlation matrix
  distance_matrix = np.dot(A_mA, A_mA.T)
  #Denominator calculation of perason correlation matrix
  distance_matrix = distance_matrix / np.sqrt(np.dot(ssA[:, None],ssA[:, None].T))

  cos_df = pd.DataFrame(distance_matrix, index=df_movie_features_new.iloc[indices[0]].index, columns=df_movie_features_new.iloc[indices[0]].index)
  return cos_df, indices[0]

def prediction(userId, movieId):
    #prediction formula implementation in whole function

    dict_with_rating = {}
    lis = []

    # Ignore movies with rating less than zero
    for idx, rating in enumerate(df_movie_features.T.loc[movieId]):
        if(rating > 0):
            temp = (idx,rating)
            dict_with_rating[idx] = rating
            lis.append(temp)

    top_k = []
    cos_df, indices=nn(userId)
    # print("In", indices)

    for i, sim in enumerate(cos_df.loc[userId]):
        for j,k in lis:
            # take only those similarities where movies have ratings greater than zero
            if(j == i):
                top_k.append([i,sim])

    sum_ = 0
    sim_sum = 0
    c=0
    for top in top_k:
      #sum over all nearest neighbour users
      # print("I", indices[c])
      mean_user = np.sum(df_movie_features_new.iloc[indices[c]])/np.count_nonzero(df_movie_features_new.iloc[indices[c]])
      sum_ + ((dict_with_rating[top[0]]-mean_user)*top[1])
      sim_sum += abs(top[1])
      c+=1

    mean_user_a = np.sum(df_movie_features_new.loc[userId])/np.count_nonzero(df_movie_features_new.loc[userId])
    # print("M",mean_user)

    if sim_sum==0 or math.isnan(sim_sum):
      return mean_user_a
    
    simil = sum_/sim_sum
    # print("simi", simil, sum_, sim_sum)
    return simil + mean_user_a



In [6]:

customer_id = TrainingRatings.customerId
# print(customer_id)
movie_id = TrainingRatings.movieId 
rating_s = TrainingRatings.rating
from sklearn.model_selection import train_test_split

ui_train, ui_test, mi_train, mi_test, r_train, r_test = train_test_split(customer_id, movie_id, rating_s, test_size=0.001, random_state = 2)
print(len(mi_test))
print(ui_test.iloc[0])


3256
2013478


In [ ]:
import warnings
import math
warnings.filterwarnings('ignore')
em_list = []
for j in range(len(mi_test)):
  print(j)
  predicted_rats = prediction(ui_test.iloc[j], mi_test.iloc[j])
  
  rating_Q = r_test.iloc[j]
  print(predicted_rats, r_test.iloc[j])
  
  half_rmse = (predicted_rats - rating_Q)**2
  em_list.append(half_rmse)

rmse = math.sqrt(sum(em_list)/len(em_list))

print("RMSE:",rmse)

In [67]:
import math
rmse = math.sqrt(sum(em_list)/len(em_list))
print("RMSE:",rmse)

RMSE: 0.9644970440335828


In [8]:
import warnings
import math
warnings.filterwarnings('ignore')
path_test="TestingRatings.txt"

with open("/content/drive/MyDrive/predictions.txt", 'a')as w:
  TestingRatings = pd.read_csv(path_test, header=None, names=['movieId', 'customerId', 'rating'])
  TestingRatings
  customer_id_test = TestingRatings.customerId
  # print(customer_id)
  movie_id_test = TestingRatings.movieId 
  rating_s_test = TestingRatings.rating.fillna(0)

  em_list = []
  y_actual=[]
  y_pred=[]
  for j in range(len(movie_id_test)):
    print("Row Count for tracking", j)
    predicted_rats = prediction(customer_id_test.iloc[j], movie_id_test.iloc[j])
    
    rating_Q = rating_s_test.iloc[j]
    print("Actual Rating", rating_s_test.iloc[j], "......Predicted Rating", predicted_rats)
    
    y_actual.append(rating_Q)
    y_pred.append(predicted_rats)
    w.write(str(rating_s_test.iloc[j])+","+ str(predicted_rats))
    w.write("\n")
    # half_rmse = (predicted_rats - rating_Q)**2
    # em_list.append(half_rmse)

  # rmse_test = math.sqrt(sum(em_list)/len(em_list))
  # print("RMSE:",rmse_test)
  MSE = np.square(np.subtract(y_actual,y_pred)).mean() 
  
  RMSE = math.sqrt(MSE)
  print("Root Mean Square Error:\n")
  print(RMSE)

Streaming output truncated to the last 5000 lines.
Actual Rating 5.0 ......Predicted Rating 3.8271604938271606
Row Count for tracking 97980
Actual Rating 5.0 ......Predicted Rating 2.888888888888889
Row Count for tracking 97981
Actual Rating 4.0 ......Predicted Rating 4.027397260273973
Row Count for tracking 97982
Actual Rating 5.0 ......Predicted Rating 4.11214953271028
Row Count for tracking 97983
Actual Rating 4.0 ......Predicted Rating 3.1625
Row Count for tracking 97984
Actual Rating 4.0 ......Predicted Rating 3.161290322580645
Row Count for tracking 97985
Actual Rating 5.0 ......Predicted Rating 4.24
Row Count for tracking 97986
Actual Rating 5.0 ......Predicted Rating 3.2790697674418605
Row Count for tracking 97987
Actual Rating 5.0 ......Predicted Rating 3.4507042253521125
Row Count for tracking 97988
Actual Rating 5.0 ......Predicted Rating 3.264
Row Count for tracking 97989
Actual Rating 5.0 ......Predicted Rating 4.209876543209877
Row Count for tracking 97990
Actual Rating 5

In [9]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_actual,y_pred)

0.7902268915281672